In [1]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [8]:
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = [1,1,1,1,1,0,0,0,0,0]

In [9]:
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[6, 26], [20, 39], [5, 2], [22, 39], [45], [9], [49, 2], [35, 20], [49, 39], [25, 43, 26, 29]]


In [10]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 6 26  0  0]
 [20 39  0  0]
 [ 5  2  0  0]
 [22 39  0  0]
 [45  0  0  0]
 [ 9  0  0  0]
 [49  2  0  0]
 [35 20  0  0]
 [49 39  0  0]
 [25 43 26 29]]


In [11]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=1)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Epoch 1/50
10/10 [==============================] - 0s - loss: 0.6909 - acc: 0.5000
Epoch 2/50
10/10 [==============================] - 0s - loss: 0.6896 - acc: 0.5000
Epoch 3/50
10/10 [==============================] - 0s - loss: 0.6882 - acc: 0.5000
Epoch 4/50
10/10 [==============================] - 0s - loss: 0.6869 - acc: 0.5000
Epoch 5/50
10/10 [==============================] - 0s - loss: 0.6855 - acc: 0.5000
Epoch 6/50
10/10 [==============================] - 0s - loss: 0.6842 - acc: 0.5000
Epoch 7/50
10/10 [==============================] - 0s - loss: 0.6829 - acc: 0.6000
Epoch 8/50
10/10 [==============================] - 0s - loss: 0.6815 - acc: 0.6000
Epoch 9/50
10/10 [==============================] - 0s - loss: 0.6802 - acc: 0.7000
Epoch 10/50
10/10 [==============================] - 0s - loss: 0.6788 - acc: 0.7000
Epoch 11/50
10/10 [==============================] - 0s - loss: 0.6775 - acc: 0.7000
Epoch 12/50
10/10 [==============================] - 0s - loss: 0.6761 - a

### Word2Vec

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, norm_only=True)
# getting word vectors of a word
dog = model['dog']
#performing king queen magic
print(model.most_similar(positive=['woman', 'king'], negative=['man']))

#picking odd one out
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

#printing similarity index
print(model.similarity('woman', 'man'))

In [ ]:
sentence=[[‘Neeraj’,’Boy’],[‘Sarwan’,’is’],[‘good’,’boy’]]
model = gensim.models.Word2Vec(sentence, min_count=1,size=300,workers=4)
print(model.similarity('woman', 'man'))